# This notebook demonstrates how a stock workpiece is beeing machined virtually

First of all, all the necessary python libraries are imported. _clear_pro_ contains my own developed functions. _spatialmath_ is a library for spatial matrices.

In [1]:
import src.clear_pro as cp
from spatialmath import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Then an ini-file is imported that contains the ideal/nominal robot model parameters. Ideal/nominal denotes the angle and lenght specifications as they are found in the robot data sheet for example.

In [ ]:
# instantiation of Parameter object
parameters = cp.Parameters()
parameters_ideal = parameters.read_parameters(
    'data/parameter/ideal_parameters.ini'
    )
parameters_ideal.pretty_print()

Following an ini-file containing the real/physical robot model parameters. Those parameters differ from the ideal/nominal parameters because of influences, such as manufacturing tolerances, assembly errors and thermal effects during operation.

In [ ]:
parameters_real = parameters.read_parameters(
    'data/parameter/real_parameters.ini'
    )
# contrast variing parameters
parameters.contrast_parameters(ideal=parameters_ideal, real=parameters_real)

Next the stock workpiece _stock.stl_ is loaded and the result file for the machined workpiece is configured (_result.stl_). Also the position of the workpiece in the robot base frame is specified.

In [ ]:
# instantiation of Workpiece object
workpiece = cp.Workpiece(
    'data/stl/stock.stl',
    'data/stl/result.stl',
    SE3([1000e-3, 0, 1000e-3])*SE3.AngleAxis(0, [0, 0, 0], unit='deg')
    )

Then the tool properties are specified.

In [ ]:
# instantiation of Tool object
tool = cp.Tool(
    diameter=9,
    flute_length=40     
)

Now the tool path is loaded. It is provided as an nc file, which was created with a CAD/CAM tool. This tool path is then transformed from the workpiece to the robot base frame using the workpiece position defined above.

In [ ]:
# instantiation of Toolpath object
toolpath = cp.Toolpath()
toolpath_w = toolpath.read_nc(
    'data/toolpath/1_circle.nc'
    )
# transform toolpath from workpiece coordinate frame to robot base frame
toolpath_rb = toolpath.transform_to_base(
    toolpath_w, workpiece
    )

Finally the simulation is prepared providing the simulation class object with all the robot model parameters, the workpiece, the tool and the tool path.

In [ ]:
# instantiation of Simulation object
simulation = cp.Simulation(
    [parameters_ideal, parameters_real],
    workpiece,
    tool,
    toolpath_rb,
    import_precision=100
    )

Finally the material removal simulation is executed.

In [ ]:
pcd, mesh, grid = simulation.do_cut(
    parameters_real,
    is_pcd=True,
    is_voxel_grid=True,
    npoints=10000,
    nvoxels=50,  
    is_delete=True,  
    is_print=True
    ) 

Here the resulting point cloud is visualized.

In [ ]:
# instantiation of Visualization object
visu = cp.Visualization(simulation, workpiece, tool)
visu.plot_pcd_mesh_grid(
    objs=[
        pcd,
        ],
    types=['solid'],
    colors=[[0.75, 0.1, 0.1]],
    cs=True,
    tool=True,
    pointsize=2,
    tool_pose=0
    )

Here the resulting mesh is visualized.

In [ ]:
visu.plot_pcd_mesh_grid(
    objs=[
        mesh,
        ],
    types=['wireframe'],
    colors=[[0.75, 0.1, 0.1]],
    cs=True,
    tool=True,
    pointsize=2,
    tool_pose=4717
    )

Here the resulting voxel grid is visualized.

In [ ]:
visu.plot_pcd_mesh_grid(
    objs=[
        grid,
        ],
    types=['grid'],
    colors=[[0.75, 0.1, 0.1]],
    cs=True,
    tool=True,
    pointsize=2,
    tool_pose=4717
    )